2025-09-26 14:55:55.765551: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-26 14:55:55.772877: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-26 14:55:55.916309: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-26 14:55:56.289279: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-26 14:55:57.235713: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

[210 211 195 212 196 180 197 181 213 165 182 198 166 214 150 183 167 199
 151 215 135 168 184 200 152 216 136 120 169 185 153 201 137 217 121 105
 170 186 154 202 138 218 122 106 171 155 187 139 203  90 219 123 107 172
 156 188  91 140 204 220 124 108  75 173 189 157  92 205 141 221 125 109
  76 174 190 206 158 222  93 142 126  60 110  77 191 207 175 223 159 143
  94 127  61 111 208 224 192  78 176 160 144  95 128 209 193  62  45 177
 112  79 161 145  96 129 194 178 162  63 113  46  80 146  97 130 179 163
 114 147  64  47  81 131  98 164  30 148 115  65  82 132  48  99 149 116
  31 133  66  83 100  49 117 134  32  84  67 101 118  50  15  33  85 102
  68 119  51 103  86  69  34  16  52 104  87  70  35  17  53  88  71  36
  54  18  89  72  37   0  55  19  73  38  56   1  20  74  39  57   2  21
  58  40  22   3  59  41  23   4  42  24   5  43  25   6  44  26   7  27
   8  28   9  29  10  11  12  13  14]
839.1048845448784 904.9703037081866 65.86541916330827
12.931761733972 22.7522516215920

2025-09-26 14:56:01.068455: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-09-26 14:56:01.069106: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[210 211 195 212 196 180 197 181 213 165 182 198 166 214 150 183 167 199
 151 215 135 168 184 200 152 216 136 120 169 185 153 201 137 217 121 105
 170 186 154 202 138 218 122 106 171 155 187 139 203  90 219 123 107 172
 156 188  91 140 204 220 124 108  75 173 189 157  92 205 141 221 125 109
  76 174 190 206 158 222  93 142 126  60 110  77 191 207 175 223 159 143
  94 127  61 111 208 224 192  78 176 160 144  95 128 209 193  62  45 177
 112  79 161 145  96 129 194 178 162  63 113  46  80 146  97 130 179 163
 114 147  64  47  81 131  98 164  30 148 115  65  82 132  48  99 149 116
  31 133  66  83 100  49 117 134  32  84  67 101 118  50  15  33  85 102
  68 119  51 103  86  69  34  16  52 104  87  70  35  17  53  88  71  36
  54  18  89  72  37   0  55  19  73  38  56   1  20  74  39  57   2  21
  58  40  22   3  59  41  23   4  42  24   5  43  25   6  44  26   7  27
   8  28   9  29  10  11  12  13  14]
839.1048845448784 904.9703037081866 65.86541916330827
12.931761733972 22.7522516215920

/tmp/ipykernel_681/1956741095.py:73: UserWarning: You passed a edgecolor/edgecolors ('#0072B2') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(t_j, r[idx], marker='x', s=s_cross, alpha=1.0,
/tmp/ipykernel_681/1956741095.py:73: UserWarning: You passed a edgecolor/edgecolors ('#D55E00') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(t_j, r[idx], marker='x', s=s_cross, alpha=1.0,


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import Polynomial
import re, os, sys
from pathlib import Path

sys.path.insert(0, str(Path("..").resolve()))
from Beta_decay import helper as beta_helper
from dipoles_exp_param import helper as dipole_helper

# ---------- global style ----------
plt.rcParams.update({
    "xtick.direction": "in", "ytick.direction": "in",
    "xtick.top": True, "ytick.right": True,
    "xtick.minor.visible": True, "ytick.minor.visible": True,
    "xtick.labelsize": 12, "ytick.labelsize": 12,
    "axes.titlesize": 14, "axes.labelsize": 14,
})

# Custom (Okabe–Ito) palettes — NOT Matplotlib default
dipole_cols = ["#0072B2", "#D55E00", "#CC79A7"]   # blue, vermillion, purple
beta_cols   = ["#0072B2", "#D55E00", "#CC79A7"]  # green, orange, sky blue

# ---------- helpers ----------
def load_train_set(path_txt):
    with open(path_txt, "r") as f:
        return [tuple(map(str, line.strip().split(","))) for line in f]

def centroid_like_point(train_set):
    arr = np.array(train_set, float)
    centroid = arr.mean(axis=0)
    idx = np.argmin(np.linalg.norm(arr - centroid, axis=1))
    return train_set[idx]

def bbox_midpoint_point(train_set):
    Amin = min(float(a) for a, b in train_set)
    Amax = max(float(a) for a, b in train_set)
    Bmin = min(float(b) for a, b in train_set)
    Bmax = max(float(b) for a, b in train_set)
    cx, cy = (Amin + Amax) / 2.0, (Bmin + Bmax) / 2.0
    return min(train_set, key=lambda t: (float(t[0]) - cx)**2 + (float(t[1]) - cy)**2)

def log_jitter(x, sigma=0.02, seed=0):
    rng = np.random.default_rng(seed)
    return np.asarray(x) * np.exp(rng.normal(0.0, sigma, size=len(x)))

def split_train_test(combined, train_set):
    mask_train = np.array([pt in train_set for pt in combined], dtype=bool)
    idx_train = np.nonzero(mask_train)[0]
    idx_test  = np.nonzero(~mask_train)[0]
    return idx_train, idx_test

def maybe_subsample(idx, max_points=None, seed=0):
    if (max_points is None) or (len(idx) <= max_points):
        return idx
    rng = np.random.default_rng(seed)
    return np.sort(rng.choice(idx, size=max_points, replace=False))

def scatter_panel(ax, times, rel, idx, *, color, marker,
                  s_circle=20, s_cross=28, lw_cross=1.2,
                  jitter_sigma=0.02, jitter_seed=0, rasterized=True, label=None):
    if len(idx) == 0: return
    t = np.asarray(times, float); r = np.asarray(rel, float); idx = np.asarray(idx)
    t_j = log_jitter(t[idx], sigma=jitter_sigma, seed=jitter_seed)
    if marker == 'o':  # Alg 1
        ax.scatter(t_j, r[idx], marker='o', s=s_circle, alpha=1.0,
                   facecolors=color, edgecolors="k", linewidths=0.4,
                   zorder=3, rasterized=rasterized, label=label)
    else:              # Alg 2 (bold ×)
        ax.scatter(t_j, r[idx], marker='x', s=s_cross, alpha=1.0,
                   edgecolors=color, linewidths=lw_cross,
                   zorder=4, rasterized=rasterized, label=label)

def add_median_guides(ax, times, rel, idx, color, ls='-'):
    if len(idx) == 0: return
    t_med = float(np.median(np.asarray(times)[idx]))
    r_med = float(np.median(np.asarray(rel)[idx]))
    ax.axvline(t_med, color=color, ls=ls, lw=1.0, alpha=0.35, zorder=1)
    ax.axhline(r_med, color=color, ls=ls, lw=1.0, alpha=0.35, zorder=1)

def tag_panel(ax, tag):
    ax.text(0.012, 0.9, tag, transform=ax.transAxes,
            va='top', ha='left', fontsize=13, fontweight='bold')

def annotate_row(ax, text):
    ax.text(0.7, 0.1, text, transform=ax.transAxes,
            ha="left", va="top", fontsize=12,
            bbox=dict(facecolor="white", edgecolor="black", alpha=0.7))

# ---------- Dipole (α_D) ----------
def compute_dipole_data():
    train_set = load_train_set("../dipoles_exp_param/train_set.txt")
    central_point = bbox_midpoint_point(train_set)

    strength_dir = '../dipoles_data_all/total_strength/'
    alphaD_dir   = '../dipoles_data_all/total_alphaD/'
    pattern = re.compile(r'^strength_([0-9.]+)_([0-9.]+)\.out$')

    combined = []
    for fname in os.listdir(strength_dir):
        m = pattern.match(fname)
        if not m: continue
        beta, alpha = m.group(1), m.group(2)
        if os.path.exists(os.path.join(alphaD_dir, f'alphaD_{beta}_{alpha}.out')):
            combined.append((alpha, beta))

    idx_train, idx_test = split_train_test(combined, train_set)

    # Alg 1
    retain = 0.6
    em1 = []
    for n in [7, 16]:
        params = np.loadtxt(f'../dipoles_exp_param/params_{n}_{retain}.txt').astype(np.float32)
        alphaD, alphaD_test, times = dipole_helper.plot_alphaD(combined, params, n, central_point, retain)
        rel = np.abs(np.array(alphaD_test) - np.array(alphaD)) / np.array(alphaD_test)
        em1.append(dict(n=n, times=np.asarray(times), rel=np.asarray(rel)))

    # Alg 2
    em2 = []
    for n in [4, 10]:
        params = np.loadtxt(f'../dipoles_exp_param/params_{n}_only_alphaD.txt').astype(np.float32)
        alphaD, alphaD_test, times = dipole_helper.plot_alphaD_simple(combined, params, n, central_point)
        rel = np.abs(np.array(alphaD_test) - np.array(alphaD)) / np.array(alphaD_test)
        em2.append(dict(n=n, times=np.asarray(times), rel=np.asarray(rel)))

    return dict(combined=combined, idx_train=idx_train, idx_test=idx_test,
                em1=em1, em2=em2)

def plot_dipole_row(ax_train, ax_test, data):
    idx_train, idx_test = data["idx_train"], data["idx_test"]
    idx_test_sub = maybe_subsample(idx_test, max_points=None, seed=11)

    # Alg 1 (filled circles)
    for i, d in enumerate(data["em1"]):
        c = dipole_cols[i]
        scatter_panel(ax_train, d["times"], d["rel"], idx_train,
                      color=c, marker='o', jitter_seed=100 + d["n"])
        add_median_guides(ax_train, d["times"], d["rel"], idx_train, c, ls='-')
        scatter_panel(ax_test,  d["times"], d["rel"], idx_test_sub,
                      color=c, marker='o', jitter_seed=120 + d["n"])
        add_median_guides(ax_test,  d["times"], d["rel"], idx_test,  c, ls='-')

    # Alg 2 (×)
    for i, d in enumerate(data["em2"]):
        c = dipole_cols[i]  # shift so it’s distinct from Alg1
        scatter_panel(ax_train, d["times"], d["rel"], idx_train,
                      color=c, marker='x', jitter_seed=200 + d["n"], lw_cross=1.4)
        add_median_guides(ax_train, d["times"], d["rel"], idx_train, c, ls='--')
        scatter_panel(ax_test,  d["times"], d["rel"], idx_test_sub,
                      color=c, marker='x', jitter_seed=220 + d["n"], lw_cross=1.4)
        add_median_guides(ax_test,  d["times"], d["rel"], idx_test,  c, ls='--')

    for ax in (ax_train, ax_test):
        ax.set_yscale('log'); #ax.grid(alpha=0.25, which='both')
    ax_train.set_ylabel(r"Relative error on $\alpha_D$")
    ax_train.yaxis.set_label_coords(-0.12, 0.0)
    #ax_train.set_title("Train region"); ax_test.set_title("Test region")
    annotate_row(ax_train, "Training set")
    annotate_row(ax_test,  "Test set")

# ---------- Beta decay (T1/2) ----------
def compute_beta_data():
    A, Z, g_A, nucnam = 80, 28, 1.2, "Ni_80"
    coeffs = Polynomial(beta_helper.fit_phase_space(0, Z, A, 15)).coef

    train_set = load_train_set("../Beta_decay/train_set.txt")
    central_point = centroid_like_point(train_set)

    strength_dir = f'../beta_decay_data_{nucnam}/'
    pattern = re.compile(rf'^lorm_{nucnam}_([0-9.]+)_([0-9.]+)\.out$')

    combined = []
    for fname in os.listdir(strength_dir):
        m = pattern.match(fname)
        if m:
            beta_val, alpha_val = m.group(1), m.group(2)
            combined.append((alpha_val, beta_val))

    idx_train, idx_test = split_train_test(combined, train_set)

    # Alg 1
    em1 = []
    for n in [7, 16]:
        params = np.loadtxt(f'../Beta_decay/params_{n}_0.9.txt')
        HLs, HLs_test, times = beta_helper.plot_half_lives(combined, params, n, coeffs, g_A, central_point, nucnam, 0.9)
        rel = np.abs(np.array(HLs_test) - np.array(HLs)) / np.array(HLs_test)
        em1.append(dict(n=n, times=np.asarray(times), rel=np.asarray(rel)))

    # Alg 2
    em2 = []
    for n in [5, 14]:
        params = np.loadtxt(f'../Beta_decay/params_{n}_only_HL.txt')
        HLs, HLs_test, times = beta_helper.plot_half_lives_only_HL(combined, params, n, coeffs, g_A, central_point, nucnam)
        rel = np.abs(np.array(HLs_test) - np.array(HLs)) / np.array(HLs_test)
        em2.append(dict(n=n, times=np.asarray(times), rel=np.asarray(rel)))

    return dict(combined=combined, idx_train=idx_train, idx_test=idx_test,
                em1=em1, em2=em2)

def plot_beta_row(ax_train, ax_test, data):
    idx_train, idx_test = data["idx_train"], data["idx_test"]
    idx_test_sub = maybe_subsample(idx_test, max_points=None, seed=21)

    # Alg 1 (filled circles)
    for i, d in enumerate(data["em1"]):
        c = beta_cols[i]
        scatter_panel(ax_train, d["times"], d["rel"], idx_train,
                      color=c, marker='o', jitter_seed=300 + d["n"])
        add_median_guides(ax_train, d["times"], d["rel"], idx_train, c, ls='-')
        scatter_panel(ax_test,  d["times"], d["rel"], idx_test_sub,
                      color=c, marker='o', jitter_seed=320 + d["n"])
        add_median_guides(ax_test,  d["times"], d["rel"], idx_test,  c, ls='-')

    # Alg 2 (×)
    for i, d in enumerate(data["em2"]):
        c = beta_cols[i]
        scatter_panel(ax_train, d["times"], d["rel"], idx_train,
                      color=c, marker='x', jitter_seed=400 + d["n"], lw_cross=1.4)
        add_median_guides(ax_train, d["times"], d["rel"], idx_train, c, ls='--')
        scatter_panel(ax_test,  d["times"], d["rel"], idx_test_sub,
                      color=c, marker='x', jitter_seed=420 + d["n"], lw_cross=1.4)
        add_median_guides(ax_test,  d["times"], d["rel"], idx_test,  c, ls='--')

    for ax in (ax_train, ax_test):
        ax.set_yscale('log'); #ax.grid(alpha=0.25, which='both')
    ax_train.set_ylabel(r"Relative error on $T_{1/2}$")
    ax_train.yaxis.set_label_coords(-0.12, 0.0)
    #ax_train.set_title("Train region"); ax_test.set_title("Test region")
    annotate_row(ax_train, "Training set")
    annotate_row(ax_test,  "Test set")

# ---------- 4×1 layout with tags and shared x ----------
def main():
    fig, axes = plt.subplots(4, 1, figsize=(5, 11.2), sharex=True,
                             gridspec_kw={"hspace": 0.0})
    ax_a, ax_b, ax_c, ax_d = axes

    dipole_data = compute_dipole_data()
    plot_dipole_row(ax_a, ax_b, dipole_data)

    beta_data = compute_beta_data()
    plot_beta_row(ax_c, ax_d, beta_data)

    # Shared x-axis for ALL panels
    x_min, x_max = 1e-4, 2e-3
    for ax in axes:
        ax.set_xscale('log'); ax.set_xlim(x_min, x_max)

    # Panel tags
    tag_panel(ax_a, "(a)"); tag_panel(ax_b, "(b)")
    tag_panel(ax_c, "(c)"); tag_panel(ax_d, "(d)")

    # Single xlabel for the whole figure
    axes[3].set_xlabel("time (s)", fontsize=14)

    # Compact legends outside axes
    from matplotlib.lines import Line2D
    handles_dip = [Line2D([], [], marker='o', linestyle='None', color=dipole_cols[i], label=f"$n_1={n}$")
                   for i, n in enumerate([7, 16])]
    handles_dip += [Line2D([], [], marker='x', linestyle='None', color=dipole_cols[i], label=f"$n_2={n}$")
                    for i, n in enumerate([4, 10])]

    axes[0].legend(handles=handles_dip, loc="upper center", ncol=5,
               bbox_to_anchor=(0.5, 1.15), frameon=False)


    plt.savefig("CATPlot.pdf", bbox_inches="tight")
    plt.subplots_adjust(hspace = 0.0)
    plt.show()

if __name__ == "__main__":
    main()


[210 211 195 212 196 180 197 181 213 165 182 198 166 214 150 183 167 199
 151 215 135 168 184 200 152 216 136 120 169 185 153 201 137 217 121 105
 170 186 154 202 138 218 122 106 171 155 187 139 203  90 219 123 107 172
 156 188  91 140 204 220 124 108  75 173 189 157  92 205 141 221 125 109
  76 174 190 206 158 222  93 142 126  60 110  77 191 207 175 223 159 143
  94 127  61 111 208 224 192  78 176 160 144  95 128 209 193  62  45 177
 112  79 161 145  96 129 194 178 162  63 113  46  80 146  97 130 179 163
 114 147  64  47  81 131  98 164  30 148 115  65  82 132  48  99 149 116
  31 133  66  83 100  49 117 134  32  84  67 101 118  50  15  33  85 102
  68 119  51 103  86  69  34  16  52 104  87  70  35  17  53  88  71  36
  54  18  89  72  37   0  55  19  73  38  56   1  20  74  39  57   2  21
  58  40  22   3  59  41  23   4  42  24   5  43  25   6  44  26   7  27
   8  28   9  29  10  11  12  13  14]
839.1048845448784 904.9703037081866 65.86541916330827
12.931761733972 22.7522516215920

Saved SplitViolins_4violins.pdf


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import Polynomial
from scipy.stats import gaussian_kde
import re, os, sys
from pathlib import Path
from matplotlib.colors import to_rgb, to_hex
from matplotlib.patches import Patch

# ---------------------------------------------------------------------
# Project imports (same as CAT plot)
# ---------------------------------------------------------------------
sys.path.insert(0, str(Path("..").resolve()))
from Beta_decay import helper as beta_helper
from dipoles_exp_param import helper as dipole_helper


from matplotlib.colors import to_rgb, to_hex



# --- emulator brand colors ---
EM1_COLOR = "#17becf"   # Cyan (Emulator 1)
EM2_COLOR = "#d62728"   # Strong red (Emulator 2)


#Helper function to change the colors
def _darken(hexcolor, amount=0.25):
    """Move color toward black by `amount` in [0,1]."""
    r, g, b = to_rgb(hexcolor)
    return to_hex((r*(1-amount), g*(1-amount), b*(1-amount)))





# ---------------------------------------------------------------------
# Global style
# ---------------------------------------------------------------------
plt.rcParams.update({
    "xtick.direction": "in", "ytick.direction": "in",
    "xtick.top": True, "ytick.right": True,
    "xtick.minor.visible": True, "ytick.minor.visible": True,
    "xtick.labelsize": 16, "ytick.labelsize": 14,
    "axes.titlesize": 14, "axes.labelsize": 16,
})



# ---------------------------------------------------------------------
# Helpers (same logic as CAT plot)
# ---------------------------------------------------------------------
def load_train_set(path_txt):
    with open(path_txt, "r") as f:
        return [tuple(map(str, line.strip().split(","))) for line in f]

def centroid_like_point(train_set):
    arr = np.array(train_set, float)
    centroid = arr.mean(axis=0)
    idx = np.argmin(np.linalg.norm(arr - centroid, axis=1))
    return train_set[idx]

def bbox_midpoint_point(train_set):
    Amin = min(float(a) for a, b in train_set)
    Amax = max(float(a) for a, b in train_set)
    Bmin = min(float(b) for a, b in train_set)
    Bmax = max(float(b) for a, b in train_set)
    cx, cy = (Amin + Amax) / 2.0, (Bmin + Bmax) / 2.0
    return min(train_set, key=lambda t: (float(t[0]) - cx)**2 + (float(t[1]) - cy)**2)

def split_train_test(combined, train_set):
    mask_train = np.array([pt in train_set for pt in combined], dtype=bool)
    idx_train = np.nonzero(mask_train)[0]
    idx_test  = np.nonzero(~mask_train)[0]
    return idx_train, idx_test

# ---------------------------------------------------------------------
# Dipole (α_D) — same as CAT logic
# ---------------------------------------------------------------------
def compute_dipole_data():
    train_set = load_train_set("../dipoles_exp_param/train_set.txt")
    central_point = bbox_midpoint_point(train_set)

    strength_dir = '../dipoles_data_all/total_strength/'
    alphaD_dir   = '../dipoles_data_all/total_alphaD/'
    pattern = re.compile(r'^strength_([0-9.]+)_([0-9.]+)\.out$')

    combined = []
    for fname in os.listdir(strength_dir):
        m = pattern.match(fname)
        if not m:
            continue
        beta, alpha = m.group(1), m.group(2)
        if os.path.exists(os.path.join(alphaD_dir, f'alphaD_{beta}_{alpha}.out')):
            combined.append((alpha, beta))

    idx_train, idx_test = split_train_test(combined, train_set)

    # Emulator 1
    retain = 0.6
    em1 = []
    for n in [7, 16]:
        params = np.loadtxt(f'../dipoles_exp_param/params_{n}_{retain}.txt').astype(np.float32)
        alphaD, alphaD_test, times = dipole_helper.plot_alphaD(combined, params, n, central_point, retain)
        rel = np.abs(np.array(alphaD_test) - np.array(alphaD)) / np.array(alphaD_test)
        em1.append(dict(n=n, times=np.asarray(times), rel=np.asarray(rel)))

    # Emulator 2
    em2 = []
    for n in [4, 10]:
        params = np.loadtxt(f'../dipoles_exp_param/params_{n}_only_alphaD.txt').astype(np.float32)
        alphaD, alphaD_test, times = dipole_helper.plot_alphaD_simple(combined, params, n, central_point)
        rel = np.abs(np.array(alphaD_test) - np.array(alphaD)) / np.array(alphaD_test)
        em2.append(dict(n=n, times=np.asarray(times), rel=np.asarray(rel)))

    return dict(combined=combined, idx_train=idx_train, idx_test=idx_test, em1=em1, em2=em2)

# ---------------------------------------------------------------------
# Beta decay (T1/2) — same as CAT logic
# ---------------------------------------------------------------------
def compute_beta_data():
    A, Z, g_A, nucnam = 80, 28, 1.2, "Ni_80"
    coeffs = Polynomial(beta_helper.fit_phase_space(0, Z, A, 15)).coef

    train_set = load_train_set("../Beta_decay/train_set.txt")
    central_point = centroid_like_point(train_set)

    strength_dir = f'../beta_decay_data_{nucnam}/'
    pattern = re.compile(rf'^lorm_{nucnam}_([0-9.]+)_([0-9.]+)\.out$')

    combined = []
    for fname in os.listdir(strength_dir):
        m = pattern.match(fname)
        if m:
            beta_val, alpha_val = m.group(1), m.group(2)
            combined.append((alpha_val, beta_val))

    idx_train, idx_test = split_train_test(combined, train_set)

    # Emulator 1
    em1 = []
    for n in [7, 16]:
        params = np.loadtxt(f'../Beta_decay/params_{n}_0.9.txt')
        HLs, HLs_test, times = beta_helper.plot_half_lives(combined, params, n, coeffs, g_A, central_point, nucnam, 0.9)
        rel = np.abs(np.array(HLs_test) - np.array(HLs)) / np.array(HLs_test)
        em1.append(dict(n=n, times=np.asarray(times), rel=np.asarray(rel)))

    # Emulator 2
    em2 = []
    for n in [5, 14]:
        params = np.loadtxt(f'../Beta_decay/params_{n}_only_HL.txt')
        HLs, HLs_test, times = beta_helper.plot_half_lives_only_HL(combined, params, n, coeffs, g_A, central_point, nucnam)
        rel = np.abs(np.array(HLs_test) - np.array(HLs)) / np.array(HLs_test)
        em2.append(dict(n=n, times=np.asarray(times), rel=np.asarray(rel)))

    return dict(combined=combined, idx_train=idx_train, idx_test=idx_test, em1=em1, em2=em2)

# ---------------------------------------------------------------------
# Split-violin machinery (log-space KDE; test half lighter; min–max whiskers)
# ---------------------------------------------------------------------
def _lighten(hexcolor, amount=0.45):
    r, g, b = to_rgb(hexcolor)
    r = 1 - (1 - r) * (1 - amount)
    g = 1 - (1 - g) * (1 - amount)
    b = 1 - (1 - b) * (1 - amount)
    return to_hex((r, g, b))

def split_violin(ax, left_vals, right_vals, xcenter, *,
                 color="#0072B2", width=0.55, kde_pts=400, logy=True,
                 pad_dex=0.15, eps=1e-16):
    """
    Split violin:
      - Left half (train) uses 'color'
      - Right half (test) uses a lighter shade of 'color'
      - KDE is performed in log10 space with padded support to avoid sharp cutoffs
      - Thin whiskers show min–max (extrema), matching your previous behavior
    """
    L = np.asarray(left_vals,  float)
    R = np.asarray(right_vals, float)
    if logy:
        L = np.clip(L, eps, None)
        R = np.clip(R, eps, None)

    allv = np.concatenate([L, R]) if (len(L) and len(R)) else (L if len(L) else R)
    if len(allv) == 0:
        return

    # KDE in log10 space
    def kde_log(vals):
        if len(vals) < 2 or np.all(vals == vals[0]):
            return None, None
        v = np.log10(vals)
        vgrid = np.linspace(v.min() - pad_dex, v.max() + pad_dex, kde_pts)
        kde = gaussian_kde(v, bw_method="scott")
        dens = kde(vgrid)
        w = (dens / dens.max()) * (width * 0.48)
        y = 10.0 ** vgrid
        return y, w

    yL, wL = kde_log(L)
    yR, wR = kde_log(R)

    # # filled halves
    # if yL is not None:
    #     ax.fill_betweenx(yL, xcenter, xcenter - wL, facecolor=color,
    #                      edgecolor="black", linewidth=1.0, alpha=0.90, zorder=2)
    # if yR is not None:
    #     ax.fill_betweenx(yR, xcenter, xcenter + wR, facecolor=_lighten(color, 0.45),
    #                      edgecolor="black", linewidth=1.0, alpha=0.85, zorder=2)

    left_col  = _darken(color, 0.22)      # make the train half ~22% darker
    right_col = _lighten(color, 0.60)     # make the test half lighter than before
    ax.fill_betweenx(yL, xcenter, xcenter - wL, facecolor=left_col,
                     edgecolor="black", linewidth=1.0, alpha=0.95, zorder=2)
    ax.fill_betweenx(yR, xcenter, xcenter + wR, facecolor=right_col,
                     edgecolor="black", linewidth=1.0, alpha=0.90, zorder=2)


    # summary overlays: IQR + median + min–max whiskers
    def _iqr(vals):
        q1, med, q3 = np.percentile(vals, [25, 50, 75])
        return q1, med, q3
    dx = width * 0.10
    for vals, xc in [(L, xcenter - dx), (R, xcenter + dx)]:
        if len(vals) == 0:
            continue
        q1, med, q3 = _iqr(vals)
        lo, hi = vals.min(), vals.max()             # min–max whiskers
        ax.plot([xc, xc], [lo, hi], color="k", lw=1.0, zorder=3)
        ax.plot([xc, xc], [q1, q3], color="k", lw=4.0, solid_capstyle="round", zorder=4)
        ax.scatter([xc], [med], s=22, facecolor="white", edgecolor="black", zorder=5)

# ---------------------------------------------------------------------
# Panel drawing (4 violins per subplot)
# ---------------------------------------------------------------------
def _collect_per_n(em_list, idx_train, idx_test):
    """Return list of (n, rel_train, rel_test) for each entry in em_list."""
    out = []
    for d in em_list:
        rel = np.asarray(d["rel"])
        out.append((d["n"], rel[idx_train], rel[idx_test]))
    return out

def _order_by_ns(records, desired_ns):
    rmap = {n: (tr, te) for (n, tr, te) in records}
    return [(n, *rmap[n]) for n in desired_ns if n in rmap]

def _draw_panel(ax, data, title, ylabel, desired_ns_em1, desired_ns_em2, logy=True):
    # collect per-n distributions
    em1_recs = _collect_per_n(data["em1"], data["idx_train"], data["idx_test"])
    em2_recs = _collect_per_n(data["em2"], data["idx_train"], data["idx_test"])
    em1 = _order_by_ns(em1_recs, desired_ns_em1)
    em2 = _order_by_ns(em2_recs, desired_ns_em2)

    # spacing and width
    VIOLIN_WIDTH = 0.55       # width of each half-violin
    WITHIN_GAP   = 0.72       # spacing between n's inside a group
    GROUP_GAP    = 1.35       # spacing between emulator groups

    x0 = 0.00
    xpos = [x0, x0 + WITHIN_GAP, x0 + GROUP_GAP, x0 + GROUP_GAP + WITHIN_GAP]
    major_centers = [(xpos[0] + xpos[1]) / 2.0, (xpos[2] + xpos[3]) / 2.0]

    # Em1
    if len(em1) >= 1:
        n, tr, te = em1[0]
        split_violin(ax, tr, te, xpos[0], color=EM1_COLOR, width=VIOLIN_WIDTH, logy=logy)
    if len(em1) >= 2:
        n, tr, te = em1[1]
        split_violin(ax, tr, te, xpos[1], color=EM1_COLOR, width=VIOLIN_WIDTH, logy=logy)

    # Em2
    if len(em2) >= 1:
        n, tr, te = em2[0]
        split_violin(ax, tr, te, xpos[2], color=EM2_COLOR, width=VIOLIN_WIDTH, logy=logy)
    if len(em2) >= 2:
        n, tr, te = em2[1]
        split_violin(ax, tr, te, xpos[3], color=EM2_COLOR, width=VIOLIN_WIDTH, logy=logy)

    # cosmetics
    ax.set_xlim(xpos[0] - 0.45, xpos[3] + 0.45)
    if logy: ax.set_yscale("log")
    ax.grid(alpha=0.18, axis="y", which="both")
    ax.set_ylabel(ylabel)
    ax.set_title(title, loc="left")

    # major group ticks (we’ll keep only on the lower subplot later)
    ax.set_xticks(major_centers, ["Emulator 1", "Emulator 2"])

    # minor n-ticks under each violin
    minor_labels = [rf"$n_1={desired_ns_em1[0]}$", rf"$n_1={desired_ns_em1[1]}$",
                    rf"$n_2={desired_ns_em2[0]}$", rf"$n_2={desired_ns_em2[1]}$"]
    ax.set_xticks(xpos, minor=True)
    ax.set_xticklabels(minor_labels, minor=True, fontsize=14)
    ax.tick_params(axis="x", which="minor", pad=2, length=0)
    # tiny separator mark between the emulator groups
    mid = 0.5 * (xpos[1] + xpos[2])  # halfway between n1=… and n2=…
    ax.plot([mid, mid], [0, -0.035], transform=ax.get_xaxis_transform(),
            color="k", lw=1)

# ---------------------------------------------------------------------
# Build the figure
# ---------------------------------------------------------------------
def make_split_violins_4(savepath="SplitViolins_4violins.pdf", logy=True):
    dip = compute_dipole_data()
    bet = compute_beta_data()

    fig, axes = plt.subplots(2, 1, figsize=(6.0, 9.2), constrained_layout=True)

    # (a) Dipole: Em1 n=[7,16]; Em2 n=[4,10]
    _draw_panel(axes[0], dip,
                # title="(a) Dipole polarizability emulator",
                title="",
                ylabel=r"Relative error on $\alpha_D$",
                desired_ns_em1=[7, 16],
                desired_ns_em2=[4, 10],
                logy=logy)

    # (b) Half-life: Em1 n=[7,16]; Em2 n=[5,14]
    _draw_panel(axes[1], bet,
                # title="(b) Half-life emulator",
                title="",
                ylabel=r"Relative error on $T_{1/2}$",
                desired_ns_em1=[7, 16],
                desired_ns_em2=[5, 14],
                logy=logy)

    # --- x-label formatting per your request ---
    # Top panel: remove "Emulator 1/2" labels (keep n labels close to axis)
    axes[0].set_xticklabels([])
    axes[0].tick_params(axis="x", which="minor", pad=2)

    # Bottom panel: keep "Emulator 1/2" (bold, spaced away from n labels)
    axes[1].tick_params(axis="x", which="major", pad=18)
    axes[1].tick_params(axis="x", which="minor", pad=2)
    for lab in axes[1].get_xticklabels(which='major'):
        lab.set_fontweight('bold')

  # (a) Legend explaining halves — emulator-agnostic greys
    dark_grey  = to_hex((0.35, 0.35, 0.35))
    light_grey = to_hex((0.80, 0.80, 0.80))
    train_patch = Patch(facecolor=dark_grey,  edgecolor="k", alpha=0.90, label="Train set (left curves)")
    test_patch  = Patch(facecolor=light_grey, edgecolor="k", alpha=0.85, label="Test set (right curves)")
    axes[0].legend(handles=[train_patch, test_patch],
                   loc="lower left", frameon=False, fontsize=14)


    # (b) Legend for emulator identity (colors)
    em1_patch = Patch(facecolor=EM1_COLOR, edgecolor="k", alpha=0.90, label="Emulator 1")
    em2_patch = Patch(facecolor=EM2_COLOR, edgecolor="k", alpha=0.90, label="Emulator 2")
    axes[1].legend(handles=[em1_patch, em2_patch],
                   loc="lower left", frameon=False, fontsize=14)
    

    fig.savefig(savepath, bbox_inches="tight")
    plt.show()
    print(f"Saved {savepath}")


plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
# ---------------------------------------------------------------------
if __name__ == "__main__":
    make_split_violins_4()
